# Why Post-Training Matters

**Or: How we teach language models to be helpful instead of just...weird**

You've probably used ChatGPT or Claude. They answer questions, help with code, refuse to write malware. Pretty useful, right?

But here's the thing: the base model—the one that comes straight out of pre-training—doesn't do any of that naturally. It just predicts the next word based on internet text.

This notebook explains the gap between "predicting text" and "being helpful," and why closing that gap matters so much.

## The Pre-Training Gap

Here's what's wild: a pre-trained language model has already learned an incredible amount. It knows grammar, facts about the world, how to write code, even how to reason through problems. It absorbed all of this from billions of words of text.

But—and this is crucial—it learned by playing a simple game: **predict the next word**.

That's it. Given some text, what word comes next?

This is powerful, but it creates a fundamental mismatch:

**Pre-training teaches:** Given text, predict what comes next  
**Users actually want:** Given a question, provide a helpful answer

See the problem? When you ask a base model "What's the capital of France?", it doesn't think "oh, they want an answer." It thinks "what text usually follows this pattern?"

Sometimes that's a good answer. Sometimes it's...not.

## What Pre-Trained Models Actually Learn (The Math)

Let's get specific. Pre-training optimizes this objective:

$$\mathcal{L}_{\text{pretrain}} = -\sum_{t} \log P(x_t | x_{<t})$$

Okay, what does that actually mean? Let's break it down symbol by symbol:

- **$\mathcal{L}_{\text{pretrain}}$** — The pre-training loss (what we're trying to minimize). Lower is better.
- **$\sum_{t}$** — Sum over all positions $t$ in the text (every word/token)
- **$x_t$** — The actual token at position $t$ (what the text really says)
- **$x_{<t}$** — All the tokens before position $t$ (the context so far)
- **$P(x_t | x_{<t})$** — The probability the model assigns to the correct next token, given the context
- **$\log$** — We take the log of that probability (math reasons—makes the optimization nicer)
- **$-$** — We negate it, so minimizing loss = maximizing the probability of correct predictions

In plain English: **make the model assign high probability to whatever word actually comes next**.

This is brilliant for learning language. But notice what's missing: there's nothing here about being helpful, truthful, or safe. The model just learns to predict **likely** continuations, not **good** ones.

**Example:** If the internet is full of text like "How to hack into..." followed by actual hacking tutorials, well, the model learns that pattern. It's just doing what it was trained to do—predict likely next words.

In [ ]:
# Let's see this in action: Base model vs. aligned model behavior

# These are simplified examples to illustrate the difference
examples = [
    {
        "prompt": "Write a poem about",
        "base": "Write a poem about nature. The trees sway gently in the breeze...",
        "base_explanation": "Continues the text (found similar patterns in training data)",
        "aligned": "I'd be happy to write a poem for you! What topic would you like?",
        "aligned_explanation": "Recognizes this as a request and responds helpfully"
    },
    {
        "prompt": "What is 2+2?",
        "base": "What is 2+2? What is 3+3? These basic math questions...",
        "base_explanation": "Continues the pattern (maybe from a quiz or forum post)",
        "aligned": "2 + 2 = 4",
        "aligned_explanation": "Understands this is a question and provides the answer"
    },
    {
        "prompt": "Can you help me with my homework?",
        "base": "Can you help me with my homework? asked Sarah nervously...",
        "base_explanation": "Writes a story (saw similar narrative text in training)",
        "aligned": "Of course! What subject is your homework in?",
        "aligned_explanation": "Recognizes request and offers to assist"
    }
]

print("=" * 70)
print("BASE MODEL vs. ALIGNED MODEL BEHAVIOR")
print("=" * 70)

for i, ex in enumerate(examples, 1):
    print(f"\n{i}. Prompt: \"{ex['prompt']}\"")
    print(f"\n   Base model says:")
    print(f"   \"{ex['base']}\"")
    print(f"   → {ex['base_explanation']}")
    
    print(f"\n   Aligned model says:")
    print(f"   \"{ex['aligned']}\"")
    print(f"   → {ex['aligned_explanation']}")
    print()

print("=" * 70)
print("Notice: Base models aren't broken—they're just optimized for a")
print("different task. Post-training teaches them to be helpful assistants.")
print("=" * 70)

## Three Problems Post-Training Solves

Post-training isn't just one thing—it addresses three distinct problems. Let's break them down.

### 1. The Format Problem

**Issue:** Pre-trained models don't know they should **respond** to questions rather than **continue** them.

When you type "What's the weather like?", a base model might continue with "What's the weather like in London? What's the weather like in Paris?" because that's a pattern it saw in training data (maybe a quiz or list).

**Solution:** Supervised Fine-Tuning (SFT)

SFT teaches the model the Q&A format using examples like:
- **User:** "What's the weather like?"
- **Assistant:** "I don't have access to real-time weather data, but I can help you understand how to check it!"

After seeing thousands of these examples, the model learns to respond instead of continue.

### 2. The Quality Problem  

**Issue:** Many valid completions exist, but humans prefer some over others.

Ask "Explain quantum computing" and you could get:
- A helpful, clear explanation ✓
- A technically correct but incomprehensible lecture ✗
- A snarky dismissal ✗
- "I don't know" (even though it clearly does) ✗

**Solution:** Reinforcement Learning from Human Feedback (RLHF) or Direct Preference Optimization (DPO)

These techniques train models using human preferences. People rate different responses, and the model learns to produce outputs humans prefer.

### 3. The Safety Problem

**Issue:** The internet contains harmful content. Models trained on it will happily generate harmful content.

This isn't the model being malicious—it's just predicting text patterns. If "How do I hack..." is often followed by hacking tutorials in the training data, the model learns that pattern.

**Solution:** Alignment training (part of RLHF/DPO)

Through post-training, models learn to:
- Refuse harmful requests
- Be honest about limitations
- Avoid generating toxic, biased, or dangerous content

Not perfectly—this is still an active area of research—but much better than base models.

## The Alignment Tax

Okay, here's the tradeoff nobody likes to talk about (but we should).

Post-training isn't free. When you train a model to be helpful and safe, you sometimes sacrifice a bit of raw capability. This is called the "alignment tax."

Think of it like this: a base model will answer **anything**. No filters, no guardrails. Just pure next-token prediction. That includes answering questions it probably shouldn't, but it also means it never refuses to help with legitimate tasks.

An aligned model is more selective. It refuses harmful requests (good!), but sometimes it also refuses things it shouldn't, or gives overly cautious answers (less good).

Here's roughly how it shakes out:

| Capability | Pre-trained Model | After Post-Training | Net Effect |
|-----------|------------------|---------------------|------------|
| Raw knowledge | +++++ | ++++ | Slight decrease |
| Following instructions | + | +++++ | Large increase |
| Safety/Refusals | - | +++++ | Large increase |
| Helpfulness | ++ | ++++ | Moderate increase |
| Creativity (sometimes) | +++++ | +++ | Can decrease |

**The good news:** Modern techniques (like DPO, which we'll implement) minimize this tradeoff. The InstructGPT paper showed that you can actually get **better** at almost everything with good post-training, not worse.

**The key insight:** A model that refuses to help you isn't very useful, but neither is one that cheerfully explains how to build a bomb. The art is finding the right balance.

## The Evidence: InstructGPT Changed Everything

In 2022, OpenAI published a paper that proved post-training really matters: InstructGPT.

The results were kind of shocking:

- A **1.3 billion parameter** InstructGPT model outperformed the **175 billion parameter** GPT-3 base model
- Users preferred InstructGPT outputs **85% of the time** (!)
- The model was more truthful and less toxic

Let that sink in: a model 130x smaller, with good post-training, beat the massive base model.

**Why?** Because being helpful matters more than being huge.

GPT-3 knew a ton—it had 175B parameters full of knowledge. But it didn't know how to be an assistant. It would:
- Continue your question instead of answering it
- Generate plausible-sounding nonsense
- Happily produce toxic content

InstructGPT, with way fewer parameters but good alignment training, learned to:
- Actually answer questions
- Admit when it didn't know something
- Refuse harmful requests

This was a watershed moment. It showed that post-training isn't just a nice-to-have—it's essential for useful AI systems.

In [ ]:
# Let's visualize the InstructGPT results

import matplotlib.pyplot as plt
import numpy as np

# These are approximate numbers from the InstructGPT paper
models = ['GPT-3\n(175B params)', 'SFT Only\n(1.3B params)', 'InstructGPT\n(1.3B params)']
human_preference = [15, 45, 85]  # Win rate vs GPT-3 baseline
model_colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(models, human_preference, color=model_colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Human Preference Win Rate (%)', fontsize=12, fontweight='bold')
ax.set_title('Post-Training Impact: Size Isn\'t Everything', fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bar, val in zip(bars, human_preference):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
            f'{val}%', ha='center', fontsize=14, fontweight='bold')

# Add annotations
ax.annotate('Base model:\nJust next-token\nprediction', 
            xy=(0, 15), xytext=(-0.3, 30),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5),
            fontsize=9, ha='center')

ax.annotate('With RLHF:\n130x smaller,\nbut way better!', 
            xy=(2, 85), xytext=(2.3, 70),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5),
            fontsize=9, ha='center', color='darkgreen', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("KEY INSIGHT")
print("=" * 70)
print("A 1.3B parameter model with RLHF beats a 175B parameter model without it.")
print("\nPost-training isn't optional—it's what makes models actually useful.")
print("=" * 70)

## What We'll Build in This Tutorial Series

Alright, enough theory. Let's talk about what you'll actually implement.

We're going to build the complete post-training pipeline—the same techniques used to create ChatGPT, Claude, and other aligned models. Here's the roadmap:

### 1. Supervised Fine-Tuning (SFT)

We'll start with the basics: teaching a model the Q&A format.

You'll learn:
- How to format instruction data (user/assistant pairs)
- Loss masking (only train on assistant responses, not user prompts)
- Why this alone isn't enough (but it's a critical first step)

**What it does:** Turns "text predictor" into "question answerer"

### 2. Reward Models

Before we can train with human feedback, we need to predict what humans prefer.

You'll learn:
- How to train a model to score responses
- Why we can't just use human ratings directly (spoiler: we need millions of comparisons)
- How one reward model can guide training on billions of examples

**What it does:** Learns to predict "will humans like this response?"

### 3. Reinforcement Learning from Human Feedback (RLHF)

The full algorithm—PPO (Proximal Policy Optimization) with a KL (Kullback-Leibler divergence) penalty.

You'll learn:
- How to use the reward model to guide training
- Why we need a KL penalty (to prevent the model from "hacking" the reward)
- The complete PPO training loop

**What it does:** Optimizes for human preferences while staying close to the original model

### 4. Direct Preference Optimization (DPO)

A simpler, more elegant alternative to RLHF that skips the reward model entirely.

You'll learn:
- How to train directly on preference pairs
- Why this works as well as RLHF (or better!)
- When to use DPO vs. RLHF

**What it does:** Same goal as RLHF, but more efficient

---

By the end, you'll understand exactly how models like GPT-4 and Claude go from "impressive but weird" to "actually useful."

Let's get started.

## Next Steps

Now that you understand **why** post-training matters, it's time to see **how** it works.

Head to the next notebook to see the project overview, then we'll dive into Supervised Fine-Tuning (SFT)—where you'll train your first aligned model.

See you there!